In [93]:
import numpy as np
import cv2 as cv

In [94]:
np.set_printoptions(suppress=True)

In [95]:
from tpk4170.visualization import Viewer
from tpk4170.models import Ball, Grid, Axes
from tpk4170.utils.transformations import quaternion_from_matrix, unit_vector

In [96]:
viewer = Viewer()
viewer.add(Grid())
viewer.add(Axes(0.5))
cam_axes = Axes(0.5)
viewer.add(cam_axes)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [97]:
camera_matrix = np.array([[750.0, 0.0, 1280/2], 
              [0.0, 750.0, 1024/2], 
              [0.0, 0.0, 1.0]])
dist_coeffs = np.zeros((5,1))

In [98]:
tvec = np.array([0.5, 0.5, 2.0])
R = np.array([[1.0, 0.0, 0.0, 0.0], 
              [0.0, -1.0, 0.0, 0.0], 
              [0.0, 0.0, -1.0, 0.0],
              [0.0,0.0,0.0,1.0]])
rvec, _ = cv.Rodrigues(R[:3,:3])

In [99]:
cam_axes.position = tvec.tolist()
cam_axes.quaternion = quaternion_from_matrix(R).tolist()

In [209]:
obj_points = np.array([[0.5, 0.5, 0.1234], 
                       [0.5, 1.5, 0.1], 
                       [1.5, 1.5, 0.2], 
                       [1.5, 0.5, 0.3]]) /2.0

In [210]:
for p in obj_points:
    ball = Ball()
    ball.position = p.tolist()
    viewer.add(ball)

In [211]:
img_points, _ = cv.projectPoints(obj_points, rvec, tvec, camera_matrix, dist_coeffs)
img_points

array([[[ 930.20275499,  608.73425166]],

       [[ 928.46153846,  415.84615385]],

       [[1133.42105263,  413.31578947]],

       [[1146.75675676,  613.35135135]]])

In [212]:
img_points = cv.undistortPoints(img_points, camera_matrix, dist_coeffs)

In [213]:
ret, rvecs, tvecs = cv.solveP3P(obj_points[:3], img_points[:3], camera_matrix, dist_coeffs, cv.SOLVEPNP_AP3P)
print(ret)
for r, t in zip(rvecs, tvecs):
    R = np.eye(4)
    R[:3,:3] = cv.Rodrigues(r)[0]
#     print(t)
#     print(R)
#     print()
    axes = Axes(0.5)
    axes.position = t.ravel().tolist()
    axes.quaternion = quaternion_from_matrix(R).tolist()
    viewer.add(axes)

2


In [214]:
ret, rvecs, tvecs = cv.solvePnP(obj_points, img_points, camera_matrix, dist_coeffs, cv.SOLVEPNP_P3P)
rvecs, tvecs

(array([[-1.54136642],
        [ 0.02849365],
        [-1.42339106]]), array([[-1211.66000999],
        [ -969.2207256 ],
        [ 1420.42503504]]))

In [215]:
def cos_angle_between_vectors(a,b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    cos_theta = np.dot(a,b)
    return cos_theta
    
def angle_between_vectors(a,b):
    return np.arctan2(np.linalg.norm(np.cross(a,b)), np.dot(a,b))

In [216]:
a, b, c = img_points[:3].reshape(-1,2)

In [217]:
b

array([ 0.38461538, -0.12820513])

In [218]:
obj_points

array([[0.25  , 0.25  , 0.0617],
       [0.25  , 0.75  , 0.05  ],
       [0.75  , 0.75  , 0.1   ],
       [0.75  , 0.25  , 0.15  ]])

In [219]:
def p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc):
    K1 = Rbc**2 / Rac**2
    K2 = Rbc**2 / Rab**2
    
    G4 = (K1*K2-K1-K2)**2 - 4*K1*K2*Cbc**2
    
    G3 = 4*(K1*K2-K1-K2)*K2*(1-K1)*Cab + 4*K1*Cbc*((K1*K2-K1+K2)*Cac + 2*K2*Cab*Cbc)
    
    G2 = (2*K2*(1-K1)*Cab)**2 +\
            2*(K1*K2-K1-K2)*(K1*K2+K1-K2) +\
            4*K1*((K1-K2)*Cbc**2 + K1*(1-K2)*Cac**2 - 2*(1+K1)*K2*Cab*Cac*Cbc)
    
    G1 = 4*(K1*K2+K1-K2)*K2*(1-K1)*Cab +\
            4*K1*((K1*K2-K1+K2)*Cac*Cbc + 2*K1*K2*Cab*Cac**2)
    
    G0 = (K1*K2+K1-K2)**2 - 4*K1**2*K2*Cac**2
    
    coeffs = (G4, G3, G2, G1, G0)
    
    xs = np.roots(coeffs)
    xs = np.real(xs[xs.imag < 1e-5])
    
    return xs

In [220]:
p3p_coeffs(2*np.sqrt(3), 2*np.sqrt(3), 2*np.sqrt(3), 20/32, 20/32., 20/32)

array([4.  , 1.  , 1.  , 0.25])

In [221]:
def p3p(obj_points, img_points, camera_matrix, dist_coeffs):
    A, B, C = obj_points.reshape(-1,3)[:3]
    
    hom_img_points = np.hstack((img_points[:3].reshape(-1,2), 
                                np.ones((3,1)))).T

    x, y, z = (np.linalg.inv(camera_matrix) @ hom_img_points).T
    
    Rab = np.linalg.norm(B-A)
    Rac = np.linalg.norm(C-A)
    Rbc = np.linalg.norm(C-B)
    
#     Rab = Rac = Rbc = 2 * np.sqrt(3)
    
    Cab = np.cos(angle_between_vectors(x,y))
    Cac = np.cos(angle_between_vectors(x,z))
    Cbc = np.cos(cos_angle_between_vectors(y,z))
    
    print(Cab, Cac, Cbc)
    
#     Cab = Cac = Cbc = 20./32.
    
    xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
    
    as_ = np.array([Rab / np.sqrt(x**2 - 2*x*Cab + 1) for x in xs])
    
#     bs = [a * x for a,x in zip(as_, xs)]
    
#     def compute_y(x, a, b):
#         m = 1-K1
#         p = 2*(K1*Cac-x*Cbc)
#         q = x**2-K1
#         m_ = 1
#         p_ = 2*(-x*Cbc)
#         q_ = (x**2*(1-K2)+2*x*K2*Cab-K2)
#         num = (p_*q - p*q_)
#         den = (m*q_ - m_*q)
#         if np.allclose(den, 0.0):
#             y = Cac + np.sqrt(Cac**2 + (Rac**2 - a**2)/a**2)
#             print(y)
#             ym = Cac - np.sqrt(Cac**2 + (Rac**2 - a**2)/a**2)
#             print(ym)
#             if np.allclose(b**2 + (y*a)**2 - 2*b*(y*a)*Cbc, Rbc**2):
#                 return y
#             else:
#                 return ym
#         else:
#             return num / den
    
#     ys = [compute_y(x, a, b) for x, a, b in zip(xs, as_, bs)] 
#     print(ys)
# #     print(ys)
    
#     cs = [y*a for y, a, b in zip(ys, as_, bs) if np.allclose(b**2 + (y*a)**2 - 2*b*(y*a)*Cbc, Rbc**2)]
    
#     ret = np.array([(x, a, b, y, c) for (x,a,b,y,c) in zip(xs, as_, bs, ys, cs)])
    
    return ret

In [222]:
p3p(obj_points, img_points, camera_matrix, dist_coeffs)

0.9999999790167016 0.9999999432462598 0.5403023230649344


True

In [223]:
hom_obj_points = np.hstack((obj_points, np.ones((4,1)))).T
hom_obj_points

array([[0.25  , 0.25  , 0.75  , 0.75  ],
       [0.25  , 0.75  , 0.75  , 0.25  ],
       [0.0617, 0.05  , 0.1   , 0.15  ],
       [1.    , 1.    , 1.    , 1.    ]])

In [224]:
img_points2 = np.eye(3,4) @ hom_obj_points
T = R.copy()
T[:3,3] = tvec
T

array([[ 0.99024041, -0.03752319,  0.13422348,  0.5       ],
       [-0.03527125, -0.99919491, -0.01911713,  0.5       ],
       [ 0.13483275,  0.01419633, -0.99076667,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [225]:
img_points2 = (np.eye(3,4) @ T @ hom_obj_points)
print(img_points2)

[[ 0.74646089  0.72612888  1.22796026  1.25343303]
 [ 0.24020393 -0.25916985 -0.27776133  0.22088027]
 [ 1.97612697  1.9948171   2.01269514  1.95605864]]


In [226]:
for u,v,w in img_points2.T:
    print(u/w, v/w, 1)

0.37773933879213584 0.12155288472235228 1
0.364007749901904 -0.12992161196335755 1
0.6101074315234711 -0.13800467202670444 1
0.6407952212212431 0.11292108662364878 1


In [227]:
img_points2 = np.array([(u/w, v/w, 1) for (u,v,w) in img_points2])
print(img_points2)

ValueError: too many values to unpack (expected 3)

In [228]:
cv.undistortPoints(img_points, camera_matrix, dist_coeffs)

array([[[-0.85281742, -0.68249469]],

       [[-0.85282051, -0.68283761]],

       [[-0.85245614, -0.68284211]],

       [[-0.85243243, -0.68248649]]])

In [229]:
np.roots((-0.5625, 3.515625, -5.90625, 3.515625, -0.5625))

array([4.  +0.j        , 1.  +0.00000005j, 1.  -0.00000005j,
       0.25+0.j        ])

In [230]:
tvec + unit_vector(tvec-x)*lengths[0] 

NameError: name 'x' is not defined

In [231]:
unit_vector(tvec - A)

array([0.12309149, 0.12309149, 0.98473193])

In [232]:
unit_vector(z)

NameError: name 'z' is not defined

In [233]:
np.linalg.norm(tvec - A)

2.03100960115899

In [234]:
a, b, c = [np.linalg.norm(tvec - p) for p in obj_points[:3]]
print(a)

1.9702809165192663


In [235]:
Rab

0.5024937810560445

In [236]:
A, B, C = obj_points.reshape(-1,3)[:3]
Rab = np.linalg.norm(B-A)
Rac = np.linalg.norm(C-A)
Rbc = np.linalg.norm(C-B)

In [237]:
Cab = (Rab**2 - a**2 - b**2) / (-2 * a * b) 
Cac = (Rac**2 - a**2 - c**2) / (-2 * a * c) 
Cbc = (Rbc**2 - b**2 - c**2) / (-2 * b * c)
np.array([Cab, Cac, Cbc])

array([0.96798663, 0.93433887, 0.96735262])

In [238]:
cosines

array([0.99999998, 0.99999994, 0.99999998])

In [239]:
print(Cab, Cac, Cbc)

0.9679866311950411 0.9343388659929425 0.9673526203026861


In [240]:
xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
as_ = Rab / np.sqrt(xs**2 - 2*xs*Cab + 1)
as_

array([1.78075526, 1.97028092, 1.98571715, 1.95653635])

In [241]:
p, q, r = np.hstack((cv.undistortPoints(img_points2, camera_matrix, dist_coeffs).reshape(-1,2)[:3], np.ones((3,1))))

error: OpenCV(3.4.3) /io/opencv/modules/imgproc/src/undistort.cpp:565: error: (-215:Assertion failed) src.isContinuous() && (src.depth() == CV_32F || src.depth() == CV_64F) && ((src.rows == 1 && src.channels() == 2) || src.cols*src.channels() == 2) in function 'undistortPoints'


In [242]:
p

array([0.75, 0.25, 0.15])

In [243]:
obj_points

array([[0.25  , 0.25  , 0.0617],
       [0.25  , 0.75  , 0.05  ],
       [0.75  , 0.75  , 0.1   ],
       [0.75  , 0.25  , 0.15  ]])

In [244]:
fx = fy = 750
cx = 640
cy = 512
inv_fx = 1. / fx
inv_fy = 1. / fy
cx_fx = cx / fx
cy_fy = cy / fy

In [245]:
img_points[0]

array([[0.38693701, 0.128979  ]])

In [246]:
ip = img_points.reshape(-1,2)

mu0, mv0 = ip[0]
mu0 = inv_fx * mu0 - cx_fx;
mv0 = inv_fy * mv0 - cy_fy;
norm = np.sqrt(mu0 * mu0 + mv0 * mv0 + 1);
mk0 = 1. / norm; mu0 *= mk0; mv0 *= mk0;

mu1, mv1 = ip[1]
mu1 = inv_fx * mu1 - cx_fx;
mv1 = inv_fy * mv1 - cy_fy;
norm = np.sqrt(mu1 * mu1 + mv1 * mv1 + 1);
mk1 = 1. / norm; mu1 *= mk1; mv1 *= mk1;

mu2, mv2 = ip[2]
mu2 = inv_fx * mu2 - cx_fx;
mv2 = inv_fy * mv2 - cy_fy;
norm = np.sqrt(mu2 * mu2 + mv2 * mv2 + 1);
mk2 = 1. / norm; mu2 *= mk2; mv2 *= mk2;

In [247]:
cosines = np.zeros(3)
cosines[0] = mu1 * mu2 + mv1 * mv2 + mk1 * mk2;
cosines[1] = mu0 * mu2 + mv0 * mv2 + mk0 * mk2;
cosines[2] = mu0 * mu1 + mv0 * mv1 + mk0 * mk1;
cosines
# Cab, Cac, Cbc = cosines

array([0.99999998, 0.99999994, 0.99999998])

In [248]:
xs = p3p_coeffs(Rab, Rac, Rbc, Cab, Cac, Cbc)
as_ = Rab / np.sqrt(xs**2 - 2*xs*Cab + 1)
as_

array([1.78075526, 1.97028092, 1.98571715, 1.95653635])

In [249]:
np.linalg.norm(tvec - obj_points, axis=1)

array([1.97028092, 1.98179212, 1.93261481, 1.88348082])

In [ ]:
np.array([mu0, mv0, mk0])

In [ ]:
unit_vector(p)

In [ ]:
np.inner(unit_vector(p), unit_vector(q))